In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras import layers, models

In [4]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [5]:
train_data_dir = '/kaggle/input/ad-vs-nonad/Data'  
train_ad_dir = '/kaggle/input/ad-vs-nonad/Data/Ad'
train_non_ad_dir = '/kaggle/input/ad-vs-nonad/Data/Non-Ad'

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

Found 5043 images belonging to 2 classes.
Found 1260 images belonging to 2 classes.


In [8]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

17225924/17225924 [==============================] - 0s 0us/step


In [9]:
for layer in base_model.layers[:-3]:
    layer.trainable = False

In [10]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5), 
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/10


I0000 00:00:1707049095.330169     114 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


157/157 [==============================] - 120s 718ms/step - loss: 0.2468 - accuracy: 0.9066 - val_loss: 0.2952 - val_accuracy: 0.9479
Epoch 2/10
157/157 [==============================] - 79s 506ms/step - loss: 0.0733 - accuracy: 0.9796 - val_loss: 0.0574 - val_accuracy: 0.9832
Epoch 3/10
157/157 [==============================] - 79s 506ms/step - loss: 0.0289 - accuracy: 0.9900 - val_loss: 0.1365 - val_accuracy: 0.9631
Epoch 4/10
157/157 [==============================] - 80s 507ms/step - loss: 0.0440 - accuracy: 0.9893 - val_loss: 0.1204 - val_accuracy: 0.9784
Epoch 5/10
157/157 [==============================] - 79s 502ms/step - loss: 0.0325 - accuracy: 0.9924 - val_loss: 0.0631 - val_accuracy: 0.9840
Epoch 6/10
157/157 [==============================] - 79s 504ms/step - loss: 0.0238 - accuracy: 0.9910 - val_loss: 0.1346 - val_accuracy: 0.9792
Epoch 7/10
157/157 [==============================] - 78s 500ms/step - loss: 0.0186 - accuracy: 0.9946 - val_loss: 0.2138 - val_accuracy: 0.

In [16]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = '/kaggle/input/testing-adddddd/ad4.jpeg' 
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0 

In [17]:
predictions = model.predict(img_array)

if predictions[0][0] > 0.5:
    print("The image is predicted to be a non-ad creative.")
else:
    print("The image is predicted to be an ad creative.")

1/1 [==============================] - 0s 21ms/step
The image is predicted to be an ad creative.


In [18]:
from tensorflow.keras.models import save_model
model.save('final_model.h5')

In [34]:
# Assuming `base_model` is the first layer in your Sequential model
base_model = model.layers[0]

# Get the names of all layers in the base model
layer_names = [layer.name for layer in base_model.layers]

# Identify the last convolutional layer
last_conv_layer_name = None
for name in reversed(layer_names):
    if 'conv' in name:
        last_conv_layer_name = name
        break

# Print the name of the last convolutional layer
print("Last Convolutional Layer:", last_conv_layer_name)
layer_names

Last Convolutional Layer: conv_pw_13_relu


['input_1',
 'conv1',
 'conv1_bn',
 'conv1_relu',
 'conv_dw_1',
 'conv_dw_1_bn',
 'conv_dw_1_relu',
 'conv_pw_1',
 'conv_pw_1_bn',
 'conv_pw_1_relu',
 'conv_pad_2',
 'conv_dw_2',
 'conv_dw_2_bn',
 'conv_dw_2_relu',
 'conv_pw_2',
 'conv_pw_2_bn',
 'conv_pw_2_relu',
 'conv_dw_3',
 'conv_dw_3_bn',
 'conv_dw_3_relu',
 'conv_pw_3',
 'conv_pw_3_bn',
 'conv_pw_3_relu',
 'conv_pad_4',
 'conv_dw_4',
 'conv_dw_4_bn',
 'conv_dw_4_relu',
 'conv_pw_4',
 'conv_pw_4_bn',
 'conv_pw_4_relu',
 'conv_dw_5',
 'conv_dw_5_bn',
 'conv_dw_5_relu',
 'conv_pw_5',
 'conv_pw_5_bn',
 'conv_pw_5_relu',
 'conv_pad_6',
 'conv_dw_6',
 'conv_dw_6_bn',
 'conv_dw_6_relu',
 'conv_pw_6',
 'conv_pw_6_bn',
 'conv_pw_6_relu',
 'conv_dw_7',
 'conv_dw_7_bn',
 'conv_dw_7_relu',
 'conv_pw_7',
 'conv_pw_7_bn',
 'conv_pw_7_relu',
 'conv_dw_8',
 'conv_dw_8_bn',
 'conv_dw_8_relu',
 'conv_pw_8',
 'conv_pw_8_bn',
 'conv_pw_8_relu',
 'conv_dw_9',
 'conv_dw_9_bn',
 'conv_dw_9_relu',
 'conv_pw_9',
 'conv_pw_9_bn',
 'conv_pw_9_relu',
 'con

In [51]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
import matplotlib.pyplot as plt

model = load_model('/kaggle/working/final_model.h5')

base_model = model.layers[0]
last_conv_layer_name = 'conv_pw_13_relu' 

img_path = '/kaggle/input/testing-adddddd/ad4.jpeg'  # Replace with the path to your image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        model.inputs, [model.get_layer('dense_4').output, model.output]
    )
#     print(grad_model)
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

predictions = model.predict(img_array)

if predictions[0][0] > 0.5:
    print("The image is predicted to be a non-ad creative.")
else:
    print("The image is predicted to be an ad creative.")
model.layers[-1].activation = None
heatmap = make_gradcam_heatmap(img_array, model, 'dense_4')

# plt.matshow(heatmap)
# plt.show()

1/1 [==============================] - 0s 478ms/step
The image is predicted to be an ad creative.


InvalidArgumentError: {{function_node __wrapped__Mean_device_/job:localhost/replica:0/task:0/device:GPU:0}} Invalid reduction dimension (2 for input with 2 dimension(s) [Op:Mean]